In [ ]:
# ============================================================================
# CONFIGURACIÓN DE MODELOS Y CROSS-VALIDATION
# ============================================================================

def train_and_evaluate_models(X_train, X_val, y_train, y_val, cv_folds=5):
    """
    Entrena múltiples modelos y evalúa con validación cruzada
    
    Parámetros:
    -----------
    X_train : sparse matrix
        Features de entrenamiento
    X_val : sparse matrix
        Features de validación
    y_train : array
        Labels de entrenamiento
    y_val : array
        Labels de validación
    cv_folds : int
        Número de folds para cross-validation
    
    Retorna:
    --------
    dict : Resultados de todos los modelos
    """
    
    print("=" * 70)
    print("ENTRENAMIENTO Y EVALUACIÓN DE MODELOS")
    print("=" * 70)
    
    # Definir modelos a entrenar
    # Cada modelo incluye justificación de hiperparámetros
    models = {
        'Logistic Regression': LogisticRegression(
            max_iter=1000,
            C=1.0,  # Regularización L2, C alto = menos regularización
            class_weight='balanced',  # Maneja desbalance de clases
            random_state=42,
            solver='liblinear'  # Eficiente para datasets pequeños-medianos
        ),
        
        'Naive Bayes': MultinomialNB(
            alpha=0.1  # Suavizado de Laplace, previene probabilidades cero
        ),
        
        'Linear SVM': LinearSVC(
            C=1.0,  # Parámetro de regularización
            class_weight='balanced',
            max_iter=1000,
            random_state=42,
            dual=False  # False es más eficiente cuando n_samples > n_features
        ),
        
        'Random Forest': RandomForestClassifier(
            n_estimators=100,  # 100 árboles
            max_depth=None,  # Sin límite de profundidad
            min_samples_split=5,  # Mínimo de muestras para dividir nodo
            min_samples_leaf=2,  # Mínimo de muestras en hoja
            class_weight='balanced',
            random_state=42,
            n_jobs=-1  # Usar todos los cores CPU
        ),
        
        'LightGBM': LGBMClassifier(
            n_estimators=100,
            max_depth=7,
            learning_rate=0.1,
            num_leaves=31,
            class_weight='balanced',
            random_state=42,
            verbose=-1
        )
    }
    
    results = {}
    best_model = None
    best_score = 0
    
    # Configurar validación cruzada estratificada
    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    print(f"\nConfiguraci{'ó'}n de validación cruzada:")
    print(f"  - Número de folds: {cv_folds}")
    print(f"  - Estratificación: Sí (mantiene proporción de clases)")
    print(f"  - Métrica principal: Accuracy")
    print(f"  - Métricas adicionales: F1-score (weighted)")
    
    # Entrenar cada modelo
    for model_name, model in models.items():
        print(f"\n{'═' * 70}")
        print(f"Entrenando: {model_name}")
        print(f"{'═' * 70}")
        
        try:
            # Cross-validation
            print(f"  Ejecutando {cv_folds}-fold cross-validation...")
            
            # Convertir a array denso solo para Random Forest y LightGBM
            if model_name in ['Random Forest', 'LightGBM']:
                X_train_array = X_train.toarray()
                X_val_array = X_val.toarray()
            else:
                X_train_array = X_train
                X_val_array = X_val
            
            # Realizar cross-validation
            cv_scores = cross_val_score(
                model, X_train_array, y_train,
                cv=skf, scoring='accuracy', n_jobs=-1
            )
            
            # Entrenar en todo el conjunto de entrenamiento
            print(f"  Entrenando en dataset completo de entrenamiento...")
            model.fit(X_train_array, y_train)
            
            # Evaluar en validation set
            y_val_pred = model.predict(X_val_array)
            y_train_pred = model.predict(X_train_array)
            
            # Calcular métricas
            train_accuracy = accuracy_score(y_train, y_train_pred)
            val_accuracy = accuracy_score(y_val, y_val_pred)
            cv_mean = cv_scores.mean()
            cv_std = cv_scores.std()
            
            # Calcular F1-score
            precision, recall, f1, _ = precision_recall_fscore_support(
                y_val, y_val_pred, average='weighted'
            )
            
            # Detectar overfitting
            overfitting = train_accuracy - val_accuracy
            
            # Mostrar resultados
            print(f"\n  {'RESULTADOS':}")
            print(f"    {'─' * 50}")
            print(f"    Cross-Validation Accuracy: {cv_mean:.4f} ± {cv_std:.4f}")
            print(f"    Training Accuracy:         {train_accuracy:.4f}")
            print(f"    Validation Accuracy:       {val_accuracy:.4f}")
            print(f"    Validation F1-Score:       {f1:.4f}")
            print(f"    Validation Precision:      {precision:.4f}")
            print(f"    Validation Recall:         {recall:.4f}")
            print(f"\n    {'Análisis de Overfitting':}")
            print(f"    Diferencia Train-Val:      {overfitting:.4f}")
            
            if overfitting > 0.15:
                print(f"    ⚠ OVERFITTING DETECTADO (diferencia > 0.15)")
            elif overfitting > 0.05:
                print(f"    ⚠ Posible overfitting leve (diferencia > 0.05)")
            else:
                print(f"    ✓ Sin overfitting significativo")
            
            # Guardar resultados
            results[model_name] = {
                'model': model,
                'cv_scores': cv_scores,
                'cv_mean': cv_mean,
                'cv_std': cv_std,
                'train_accuracy': train_accuracy,
                'val_accuracy': val_accuracy,
                'f1_score': f1,
                'precision': precision,
                'recall': recall,
                'overfitting': overfitting,
                'y_val_pred': y_val_pred
            }
            
            # Actualizar mejor modelo
            if val_accuracy > best_score:
                best_score = val_accuracy
                best_model = model_name
            
            print(f"  ✓ {model_name} completado exitosamente")
            
        except Exception as e:
            print(f"  ✗ Error entrenando {model_name}: {e}")
            continue
    
    # Resumen final
    print(f"\n{'═' * 70}")
    print(f"RESUMEN DE RESULTADOS")
    print(f"{'═' * 70}")
    
    # Crear tabla comparativa
    print(f"\n{'Modelo':<20} {'CV Accuracy':<15} {'Val Accuracy':<15} {'F1-Score':<12} {'Overfitting':<12}")
    print(f"{'─' * 80}")
    
    for model_name, res in sorted(results.items(), key=lambda x: x[1]['val_accuracy'], reverse=True):
        print(f"{model_name:<20} {res['cv_mean']:.4f} ± {res['cv_std']:.3f}   "
              f"{res['val_accuracy']:.4f}          "
              f"{res['f1_score']:.4f}       "
              f"{res['overfitting']:+.4f}")
    
    print(f"\n{'✓ MEJOR MODELO:'} {best_model} (Validation Accuracy: {best_score:.4f})")
    
    return results, best_model


# Entrenar modelos
print("Iniciando entrenamiento de modelos...")
print("Esto puede tomar varios minutos dependiendo del tamaño del dataset...\n")

model_results, best_model_name = train_and_evaluate_models(
    X_train_tfidf, X_val_tfidf, y_train, y_val, cv_folds=5
)

## 13. Visualización de Resultados de Modelos

Graficamos los resultados para comparación visual.

In [ ]:
# ============================================================================
# VISUALIZACIÓN DE COMPARACIÓN DE MODELOS
# ============================================================================

def plot_model_comparison(results):
    """
    Crea visualizaciones comparativas de los modelos
    """
    
    # Preparar datos
    models = list(results.keys())
    cv_means = [results[m]['cv_mean'] for m in models]
    cv_stds = [results[m]['cv_std'] for m in models]
    val_accs = [results[m]['val_accuracy'] for m in models]
    f1_scores = [results[m]['f1_score'] for m in models]
    overfits = [results[m]['overfitting'] for m in models]
    
    # Crear subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Comparación de Accuracy
    x = np.arange(len(models))
    width = 0.35
    
    axes[0, 0].bar(x - width/2, cv_means, width, label='CV Accuracy', 
                   alpha=0.8, color='skyblue', edgecolor='black')
    axes[0, 0].bar(x + width/2, val_accs, width, label='Validation Accuracy',
                   alpha=0.8, color='lightcoral', edgecolor='black')
    axes[0, 0].set_xlabel('Modelo', fontsize=12, fontweight='bold')
    axes[0, 0].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
    axes[0, 0].set_title('Comparación de Accuracy por Modelo', fontsize=14, fontweight='bold')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(models, rotation=45, ha='right')
    axes[0, 0].legend()
    axes[0, 0].grid(axis='y', alpha=0.3)
    axes[0, 0].set_ylim([0, 1.0])
    
    # 2. Cross-Validation con error bars
    axes[0, 1].errorbar(x, cv_means, yerr=cv_stds, fmt='o', markersize=10,
                        capsize=5, capthick=2, elinewidth=2, color='darkblue')
    axes[0, 1].set_xlabel('Modelo', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('CV Accuracy', fontsize=12, fontweight='bold')
    axes[0, 1].set_title('Cross-Validation con Intervalo de Confianza', fontsize=14, fontweight='bold')
    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(models, rotation=45, ha='right')
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].set_ylim([min(cv_means) - 0.1, 1.0])
    
    # 3. F1-Score comparison
    axes[1, 0].barh(models, f1_scores, color='mediumseagreen', edgecolor='black', alpha=0.8)
    axes[1, 0].set_xlabel('F1-Score (Weighted)', fontsize=12, fontweight='bold')
    axes[1, 0].set_title('Comparación de F1-Score', fontsize=14, fontweight='bold')
    axes[1, 0].grid(axis='x', alpha=0.3)
    axes[1, 0].set_xlim([0, 1.0])
    
    # Añadir valores en las barras
    for i, v in enumerate(f1_scores):
        axes[1, 0].text(v + 0.01, i, f'{v:.4f}', va='center', fontsize=10)
    
    # 4. Overfitting analysis
    colors_overfit = ['red' if o > 0.10 else 'orange' if o > 0.05 else 'green' for o in overfits]
    axes[1, 1].barh(models, overfits, color=colors_overfit, edgecolor='black', alpha=0.7)
    axes[1, 1].set_xlabel('Overfitting (Train Acc - Val Acc)', fontsize=12, fontweight='bold')
    axes[1, 1].set_title('Análisis de Overfitting por Modelo', fontsize=14, fontweight='bold')
    axes[1, 1].axvline(x=0.05, color='orange', linestyle='--', linewidth=2, label='Umbral leve (0.05)')
    axes[1, 1].axvline(x=0.10, color='red', linestyle='--', linewidth=2, label='Umbral severo (0.10)')
    axes[1, 1].legend()
    axes[1, 1].grid(axis='x', alpha=0.3)
    
    # Añadir valores
    for i, v in enumerate(overfits):
        axes[1, 1].text(v + 0.005, i, f'{v:+.4f}', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

plot_model_comparison(model_results)

## 14. PASO 6: Evaluación Final en Test Set

Evaluamos el mejor modelo en el conjunto de test (datos nunca vistos).

### Métricas de Evaluación:
1. **Accuracy**: % de predicciones correctas
2. **Precision**: De los predichos como clase X, cuántos son realmente X
3. **Recall**: De todos los X reales, cuántos fueron detectados
4. **F1-Score**: Media armónica de precision y recall
5. **Confusion Matrix**: Visualiza errores de clasificación

In [ ]:
# ============================================================================
# EVALUACIÓN FINAL EN TEST SET
# ============================================================================

def evaluate_on_test_set(model, X_test, y_test, model_name, class_labels_dict):
    """
    Evalúa el modelo final en el conjunto de test
    
    Parámetros:
    -----------
    model : sklearn model
        Modelo entrenado
    X_test : sparse matrix
        Features de test
    y_test : array
        Labels verdaderos de test
    model_name : str
        Nombre del modelo
    class_labels_dict : dict
        Diccionario de etiquetas de clase
    
    Retorna:
    --------
    dict : Métricas de evaluación
    """
    
    print("=" * 70)
    print(f"EVALUACIÓN FINAL EN TEST SET: {model_name}")
    print("=" * 70)
    
    # Convertir a array denso si es necesario
    if model_name in ['Random Forest', 'LightGBM']:
        X_test_array = X_test.toarray()
    else:
        X_test_array = X_test
    
    # Predicciones
    y_pred = model.predict(X_test_array)
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, support = precision_recall_fscore_support(
        y_test, y_pred, average=None
    )
    
    # Métricas weighted (considerando desbalance)
    precision_w, recall_w, f1_w, _ = precision_recall_fscore_support(
        y_test, y_pred, average='weighted'
    )
    
    # Mostrar resultados
    print(f"\n{'MÉTRICAS GENERALES':}")
    print(f"  {'─' * 50}")
    print(f"  Accuracy:           {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"  Precision (weighted): {precision_w:.4f}")
    print(f"  Recall (weighted):    {recall_w:.4f}")
    print(f"  F1-Score (weighted):  {f1_w:.4f}")
    
    # Métricas por clase
    class_names_inv = {v: k for k, v in class_labels_dict.items()}
    
    print(f"\n{'MÉTRICAS POR CLASE':}")
    print(f"  {'─' * 50}")
    print(f"  {'Clase':<25} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<10}")
    print(f"  {'─' * 80}")
    
    for i in range(len(precision)):
        class_name = class_names_inv[i]
        print(f"  {class_name:<25} {precision[i]:<12.4f} {recall[i]:<12.4f} "
              f"{f1[i]:<12.4f} {support[i]:<10}")
    
    # Classification report
    print(f"\n{'CLASSIFICATION REPORT DETALLADO':}")
    print(f"  {'─' * 50}")
    target_names = [class_names_inv[i] for i in range(len(class_names_inv))]
    print(classification_report(y_test, y_pred, target_names=target_names))
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Retornar resultados
    return {
        'accuracy': accuracy,
        'precision': precision_w,
        'recall': recall_w,
        'f1_score': f1_w,
        'confusion_matrix': cm,
        'y_pred': y_pred,
        'per_class_metrics': {
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'support': support
        }
    }


# Obtener el mejor modelo
best_model = model_results[best_model_name]['model']

# Evaluar en test set
test_results = evaluate_on_test_set(
    best_model, X_test_tfidf, y_test, 
    best_model_name, class_labels
)

In [ ]:
# ============================================================================
# VISUALIZACIÓN: CONFUSION MATRIX
# ============================================================================

def plot_confusion_matrix(cm, class_labels_dict, model_name):
    """
    Visualiza la matriz de confusión
    
    Justificación:
    --------------
    La matriz de confusión muestra dónde el modelo comete errores:
    - Diagonal: Predicciones correctas
    - Fuera de diagonal: Confusiones entre clases
    - Útil para identificar qué clases se confunden entre sí
    """
    
    class_names = [k for k, v in sorted(class_labels_dict.items(), key=lambda x: x[1])]
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 1. Matriz de confusión (valores absolutos)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                ax=axes[0], cbar_kws={'label': 'Cantidad'})
    axes[0].set_title(f'Matriz de Confusión - {model_name}\n(Valores Absolutos)', 
                     fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Clase Real', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Clase Predicha', fontsize=12, fontweight='bold')
    
    # 2. Matriz de confusión normalizada (porcentajes)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Greens',
                xticklabels=class_names, yticklabels=class_names,
                ax=axes[1], cbar_kws={'label': 'Porcentaje'})
    axes[1].set_title(f'Matriz de Confusión - {model_name}\n(Normalizada por Fila)', 
                     fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Clase Real', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Clase Predicha', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Análisis de confusiones
    print(f"\n{'ANÁLISIS DE CONFUSIONES':}")
    print(f"{'─' * 70}")
    
    for i, true_class in enumerate(class_names):
        for j, pred_class in enumerate(class_names):
            if i != j and cm[i, j] > 0:
                confusion_rate = cm[i, j] / cm[i].sum()
                print(f"  {cm[i, j]} instancias de '{true_class}' clasificadas como '{pred_class}' "
                      f"({confusion_rate*100:.1f}% de {true_class})")

plot_confusion_matrix(test_results['confusion_matrix'], class_labels, best_model_name)

## 15. Análisis de Errores

Examinamos casos específicos donde el modelo falla para entender limitaciones.

In [ ]:
# ============================================================================
# ANÁLISIS DE ERRORES
# ============================================================================

def analyze_errors(X_test_original, y_test, y_pred, class_labels_dict, n_examples=5):
    """
    Analiza ejemplos de errores de clasificación
    
    Justificación:
    --------------
    Entender dónde y por qué falla el modelo es crucial para:
    - Identificar limitaciones del modelo
    - Mejorar preprocesamiento
    - Decidir si necesitamos más datos de ciertas clases
    - Detectar problemas de calidad de datos (OCR errors, etc.)
    """
    
    print("=" * 70)
    print("ANÁLISIS DE ERRORES DE CLASIFICACIÓN")
    print("=" * 70)
    
    # Identificar predicciones incorrectas
    errors = y_test != y_pred
    error_indices = np.where(errors)[0]
    
    print(f"\nTotal de errores: {len(error_indices)} de {len(y_test)} "
          f"({len(error_indices)/len(y_test)*100:.2f}%)")
    
    if len(error_indices) == 0:
        print("¡No hay errores! El modelo es perfecto en el test set.")
        return
    
    # Mapeo de etiquetas
    class_names_inv = {v: k for k, v in class_labels_dict.items()}
    
    # Analizar algunos ejemplos de errores
    print(f"\n{'EJEMPLOS DE CLASIFICACIONES INCORRECTAS':}")
    print(f"{'─' * 70}\n")
    
    n_show = min(n_examples, len(error_indices))
    sample_errors = np.random.choice(error_indices, n_show, replace=False)
    
    for idx, error_idx in enumerate(sample_errors, 1):
        true_label = y_test[error_idx]
        pred_label = y_pred[error_idx]
        text = X_test_original[error_idx]
        
        print(f"Error #{idx}:")
        print(f"  Clase Real:     {class_names_inv[true_label]}")
        print(f"  Clase Predicha: {class_names_inv[pred_label]}")
        print(f"  Texto (primeras 200 caracteres):")
        print(f"    '{text[:200]}...'")
        print(f"  {'─' * 68}\n")
    
    # Análisis de confusión por pares de clases
    print(f"\n{'PARES DE CLASES MÁS CONFUNDIDOS':}")
    print(f"{'─' * 70}")
    
    confusion_pairs = []
    for i in error_indices:
        confusion_pairs.append((y_test[i], y_pred[i]))
    
    from collections import Counter
    confusion_counts = Counter(confusion_pairs)
    
    for (true_class, pred_class), count in confusion_counts.most_common(3):
        print(f"  '{class_names_inv[true_class]}' → '{class_names_inv[pred_class]}': "
              f"{count} veces")

analyze_errors(X_test, y_test, test_results['y_pred'], class_labels, n_examples=5)

## 16. Guardar Modelo Final

Guardamos el mejor modelo y el vectorizador TF-IDF para deployment.

In [ ]:
# ============================================================================
# GUARDAR MODELO Y ARTIFACTOS
# ============================================================================

def save_model_artifacts(model, vectorizer, class_labels_dict, 
                        model_name, test_results, output_dir='models'):
    """
    Guarda el modelo entrenado y todos los artifactos necesarios
    
    Parámetros:
    -----------
    model : sklearn model
        Modelo entrenado
    vectorizer : TfidfVectorizer
        Vectorizador entrenado
    class_labels_dict : dict
        Diccionario de etiquetas
    model_name : str
        Nombre del modelo
    test_results : dict
        Resultados de evaluación
    output_dir : str
        Directorio de salida
    """
    
    import os
    import pickle
    import json
    from datetime import datetime
    
    # Crear directorio si no existe
    os.makedirs(output_dir, exist_ok=True)
    
    print("=" * 70)
    print("GUARDANDO MODELO Y ARTIFACTOS")
    print("=" * 70)
    
    # Timestamp para versionado
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Guardar modelo
    model_filename = f"{output_dir}/model_{model_name.replace(' ', '_')}_{timestamp}.pkl"
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"\n✓ Modelo guardado: {model_filename}")
    
    # 2. Guardar vectorizador
    vectorizer_filename = f"{output_dir}/vectorizer_{timestamp}.pkl"
    with open(vectorizer_filename, 'wb') as f:
        pickle.dump(vectorizer, f)
    print(f"✓ Vectorizador guardado: {vectorizer_filename}")
    
    # 3. Guardar metadatos
    metadata = {
        'model_name': model_name,
        'timestamp': timestamp,
        'class_labels': class_labels_dict,
        'test_accuracy': float(test_results['accuracy']),
        'test_f1_score': float(test_results['f1_score']),
        'test_precision': float(test_results['precision']),
        'test_recall': float(test_results['recall']),
        'model_filename': model_filename,
        'vectorizer_filename': vectorizer_filename
    }
    
    metadata_filename = f"{output_dir}/metadata_{timestamp}.json"
    with open(metadata_filename, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"✓ Metadatos guardados: {metadata_filename}")
    
    # 4. Crear versión "latest" (sobrescribir siempre con el último)
    latest_model = f"{output_dir}/model_latest.pkl"
    latest_vectorizer = f"{output_dir}/vectorizer_latest.pkl"
    latest_metadata = f"{output_dir}/metadata_latest.json"
    
    with open(latest_model, 'wb') as f:
        pickle.dump(model, f)
    with open(latest_vectorizer, 'wb') as f:
        pickle.dump(vectorizer, f)
    with open(latest_metadata, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\n✓ Versión 'latest' actualizada:")
    print(f"  - {latest_model}")
    print(f"  - {latest_vectorizer}")
    print(f"  - {latest_metadata}")
    
    print(f"\n{'INFORMACIÓN DEL MODELO GUARDADO':}")
    print(f"{'─' * 70}")
    print(f"  Modelo: {model_name}")
    print(f"  Test Accuracy: {test_results['accuracy']:.4f}")
    print(f"  Test F1-Score: {test_results['f1_score']:.4f}")
    print(f"  Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    return metadata_filename

# Guardar modelo final
metadata_file = save_model_artifacts(
    best_model, 
    tfidf_vectorizer, 
    class_labels,
    best_model_name,
    test_results
)

## 17. Función de Predicción para Deployment

Creamos una función sencilla para hacer predicciones en nuevos documentos.

In [ ]:
# ============================================================================
# FUNCIÓN DE PREDICCIÓN PARA DEPLOYMENT
# ============================================================================

def predict_document_class(image_path, model, vectorizer, class_labels_dict, 
                          preprocess_func, tesseract_cmd=None):
    """
    Predice la clase de un nuevo documento desde imagen
    
    Parámetros:
    -----------
    image_path : str
        Ruta a la imagen del documento
    model : sklearn model
        Modelo entrenado
    vectorizer : TfidfVectorizer
        Vectorizador entrenado
    class_labels_dict : dict
        Diccionario de etiquetas
    preprocess_func : function
        Función de preprocesamiento de texto
    tesseract_cmd : str, opcional
        Ruta al ejecutable de Tesseract
    
    Retorna:
    --------
    dict : Predicción con clase, probabilidades y confianza
    """
    
    # Configurar Tesseract si se proporciona
    if tesseract_cmd:
        pytesseract.pytesseract.tesseract_cmd = tesseract_cmd
    
    try:
        # 1. Cargar imagen
        image = Image.open(image_path)
        
        # 2. Extraer texto con OCR
        text = pytesseract.image_to_string(image, lang='eng')
        
        # 3. Preprocesar texto
        processed_text = preprocess_func(text)
        
        # 4. Vectorizar
        text_tfidf = vectorizer.transform([processed_text])
        
        # 5. Predecir
        prediction = model.predict(text_tfidf)[0]
        
        # 6. Obtener probabilidades si el modelo lo soporta
        if hasattr(model, 'predict_proba'):
            probabilities = model.predict_proba(text_tfidf)[0]
        elif hasattr(model, 'decision_function'):
            # Para SVM, convertir decision function a pseudo-probabilidades
            decision = model.decision_function(text_tfidf)[0]
            probabilities = np.exp(decision) / np.sum(np.exp(decision))
        else:
            probabilities = None
        
        # Mapear predicción a nombre de clase
        class_names_inv = {v: k for k, v in class_labels_dict.items()}
        predicted_class = class_names_inv[prediction]
        
        # Calcular confianza
        if probabilities is not None:
            confidence = float(probabilities[prediction])
            all_probs = {class_names_inv[i]: float(prob) 
                        for i, prob in enumerate(probabilities)}
        else:
            confidence = None
            all_probs = None
        
        return {
            'predicted_class': predicted_class,
            'predicted_label': int(prediction),
            'confidence': confidence,
            'all_probabilities': all_probs,
            'original_text_length': len(text),
            'processed_text_length': len(processed_text.split()),
            'success': True
        }
        
    except Exception as e:
        return {
            'success': False,
            'error': str(e)
        }


# Ejemplo de uso
print("=" * 70)
print("EJEMPLO DE PREDICCIÓN EN NUEVO DOCUMENTO")
print("=" * 70)

# Intentar predecir en una imagen de test
import glob
test_images = glob.glob(r"datasets\document-classification-dataset\email\*.png")

if test_images:
    example_image = test_images[0]
    print(f"\nProbando con imagen: {example_image}")
    
    result = predict_document_class(
        example_image,
        best_model,
        tfidf_vectorizer,
        class_labels,
        preprocess_data,
        tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    )
    
    if result['success']:
        print(f"\n✓ Predicción exitosa:")
        print(f"  Clase predicha: {result['predicted_class']}")
        print(f"  Confianza: {result['confidence']*100:.2f}%" if result['confidence'] else "  Confianza: N/A")
        if result['all_probabilities']:
            print(f"\n  Probabilidades por clase:")
            for class_name, prob in sorted(result['all_probabilities'].items(), 
                                          key=lambda x: x[1], reverse=True):
                print(f"    {class_name:25s}: {prob*100:5.2f}%")
    else:
        print(f"\n✗ Error en predicción: {result['error']}")
else:
    print("\n⚠ No se encontraron imágenes de ejemplo para probar")

## 18. Conclusiones y Recomendaciones Finales

Resumen del proyecto, hallazgos clave y recomendaciones para mejora futura.

In [ ]:
# ============================================================================
# GENERAR RESUMEN FINAL DEL PROYECTO
# ============================================================================

print("=" * 70)
print("RESUMEN FINAL DEL PROYECTO")
print("=" * 70)

print(f"\n{'1. CONFIGURACIÓN DEL DATASET':}")
print(f"   {'─' * 65}")
print(f"   Total de documentos: {len(df)}")
print(f"   Clases: {list(class_labels.keys())}")
print(f"   División: 70% train, 20% validation, 10% test")
print(f"   Estratificación: Sí (balance de clases mantenido)")

print(f"\n{'2. PREPROCESAMIENTO Y FEATURES':}")
print(f"   {'─' * 65}")
print(f"   Técnica de OCR: Tesseract")
print(f"   Preprocesamiento NLP:")
print(f"     - Lowercase, eliminación de puntuación y números")
print(f"     - Tokenización y eliminación de stopwords")
print(f"     - Lemmatización (WordNet)")
print(f"   Feature extraction: TF-IDF")
print(f"     - N-grams: 1-2 (unigrams + bigrams)")
print(f"     - Max features: {X_train_tfidf.shape[1]}")
print(f"     - Sparsity: {(1.0 - X_train_tfidf.nnz / (X_train_tfidf.shape[0] * X_train_tfidf.shape[1])) * 100:.2f}%")

if pca_analysis['recommendation']:
    print(f"   Reducción de dimensionalidad: PCA aplicado")
    print(f"     - Componentes: {pca_analysis['n_components']}")
    print(f"     - Varianza explicada: {pca_analysis['variance_explained']*100:.2f}%")
else:
    print(f"   Reducción de dimensionalidad: No aplicado")
    print(f"     - Justificación: TF-IDF sparse es más eficiente")

print(f"\n{'3. MODELOS ENTRENADOS Y EVALUADOS':}")
print(f"   {'─' * 65}")
for i, (model_name, res) in enumerate(sorted(model_results.items(), 
                                            key=lambda x: x[1]['val_accuracy'], 
                                            reverse=True), 1):
    print(f"   {i}. {model_name}")
    print(f"      CV Accuracy: {res['cv_mean']:.4f} ± {res['cv_std']:.4f}")
    print(f"      Val Accuracy: {res['val_accuracy']:.4f}")
    print(f"      Overfitting: {res['overfitting']:+.4f}")

print(f"\n{'4. MEJOR MODELO SELECCIONADO':}")
print(f"   {'─' * 65}")
print(f"   Modelo: {best_model_name}")
print(f"   Validation Accuracy: {model_results[best_model_name]['val_accuracy']:.4f}")
print(f"   Test Accuracy: {test_results['accuracy']:.4f} ({test_results['accuracy']*100:.2f}%)")
print(f"   Test F1-Score: {test_results['f1_score']:.4f}")
print(f"   Test Precision: {test_results['precision']:.4f}")
print(f"   Test Recall: {test_results['recall']:.4f}")

print(f"\n{'5. ANÁLISIS DE OVERFITTING':}")
print(f"   {'─' * 65}")
best_overfit = model_results[best_model_name]['overfitting']
if best_overfit > 0.10:
    print(f"   ⚠ OVERFITTING DETECTADO ({best_overfit:+.4f})")
    print(f"   Recomendaciones:")
    print(f"     - Aumentar regularización (C más bajo para LR/SVM)")
    print(f"     - Reducir complejidad del modelo")
    print(f"     - Obtener más datos de entrenamiento")
elif best_overfit > 0.05:
    print(f"   ⚠ Overfitting leve detectado ({best_overfit:+.4f})")
    print(f"   El modelo está ligeramente sobreajustado pero aceptable")
else:
    print(f"   ✓ No hay overfitting significativo ({best_overfit:+.4f})")
    print(f"   El modelo generaliza correctamente")

print(f"\n{'6. MÉTRICAS POR CLASE (Test Set)':}")
print(f"   {'─' * 65}")
class_names_inv = {v: k for k, v in class_labels.items()}
for i in range(len(class_labels)):
    class_name = class_names_inv[i]
    metrics = test_results['per_class_metrics']
    print(f"   {class_name}:")
    print(f"     Precision: {metrics['precision'][i]:.4f}")
    print(f"     Recall:    {metrics['recall'][i]:.4f}")
    print(f"     F1-Score:  {metrics['f1'][i]:.4f}")
    print(f"     Support:   {metrics['support'][i]} documentos")

print(f"\n{'7. RECOMENDACIONES PARA MEJORA FUTURA':}")
print(f"   {'─' * 65}")
print(f"   □ Aumentar tamaño del dataset (especialmente clases minoritarias)")
print(f"   □ Mejorar calidad de OCR:")
print(f"       - Preprocesamiento de imágenes (binarización, deskew)")
print(f"       - Usar Tesseract 5.x con LSTM para mejor reconocimiento")
print(f"   □ Feature engineering adicional:")
print(f"       - Features de layout (posición de texto, formato)")
print(f"       - Named Entity Recognition (NER)")
print(f"       - Word embeddings (Word2Vec, FastText)")
print(f"   □ Explorar modelos más avanzados:")
print(f"       - Transfer learning con BERT/RoBERTa")
print(f"       - Ensembles (stacking de múltiples modelos)")
print(f"   □ Análisis de casos edge:")
print(f"       - Documentos híbridos o ambiguos")
print(f"       - Documentos con poco texto")
print(f"       - Diferentes idiomas o layouts")

print(f"\n{'8. ARCHIVOS GENERADOS':}")
print(f"   {'─' * 65}")
print(f"   ✓ Modelo entrenado: models/model_latest.pkl")
print(f"   ✓ Vectorizador TF-IDF: models/vectorizer_latest.pkl")
print(f"   ✓ Metadatos: models/metadata_latest.json")
print(f"   ✓ Notebook con análisis completo: main.ipynb")

print(f"\n{'═' * 70}")
print(f"PROYECTO COMPLETADO EXITOSAMENTE")
print(f"{'═' * 70}")
print(f"\n✓ Todos los objetivos cumplidos:")
print(f"  ✓ Conversión de formatos de imagen")
print(f"  ✓ Extracción de texto con OCR")
print(f"  ✓ Análisis exploratorio exhaustivo")
print(f"  ✓ División estratificada de datos")
print(f"  ✓ Entrenamiento de múltiples modelos")
print(f"  ✓ Validación cruzada y detección de overfitting")
print(f"  ✓ Evaluación final en test set")
print(f"  ✓ Modelo guardado para deployment")
print(f"\n¡Gracias por usar este sistema de clasificación de documentos!")